In [1]:
"""
Real-time Jetbot tracking with temporal smoothing and improved color identification
Compatible with Python 3.6 (Jetbot environment)
"""

# Python 3.6 compatible imports
import cv2
import numpy as np
from collections import deque, defaultdict
import time
from pathlib import Path
import json

# For Python 3.6 compatibility
try:
    from typing import Dict, List, Tuple, Optional, Any
except ImportError:
    # Python 3.6 might have issues with some typing imports
    pass


class KalmanTracker:
    """
    Kalman filter for smooth tracking of robot positions and orientations
    Handles temporal consistency across frames
    """
    
    def __init__(self, dt=1.0/30.0):
        """Initialize Kalman filter for 2D position + orientation tracking"""
        self.dt = dt
        
        # State: [x, y, vx, vy, theta, omega]
        self.kf = cv2.KalmanFilter(6, 3)
        
        # State transition matrix
        self.kf.transitionMatrix = np.array([
            [1, 0, dt, 0,  0, 0],
            [0, 1, 0,  dt, 0, 0],
            [0, 0, 1,  0,  0, 0],
            [0, 0, 0,  1,  0, 0],
            [0, 0, 0,  0,  1, dt],
            [0, 0, 0,  0,  0, 1]
        ], dtype=np.float32)
        
        # Measurement matrix
        self.kf.measurementMatrix = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0]
        ], dtype=np.float32)
        
        # Process noise
        self.kf.processNoiseCov = np.eye(6, dtype=np.float32) * 0.03
        
        # Measurement noise
        self.kf.measurementNoiseCov = np.eye(3, dtype=np.float32) * 0.1
        
        # Error covariance
        self.kf.errorCovPost = np.eye(6, dtype=np.float32)
        
        # Initial state
        self.initialized = False
        
    def init_state(self, x, y, theta):
        """Initialize tracker with first measurement"""
        self.kf.statePre = np.array([x, y, 0, 0, theta, 0], dtype=np.float32)
        self.kf.statePost = self.kf.statePre.copy()
        self.initialized = True
        
    def predict(self):
        """Predict next state"""
        if self.initialized:
            return self.kf.predict()
        return None
        
    def update(self, x, y, theta):
        """Update with measurement"""
        if not self.initialized:
            self.init_state(x, y, theta)
            return self.kf.statePost
        
        measurement = np.array([x, y, theta], dtype=np.float32)
        self.kf.correct(measurement)
        return self.kf.statePost


class ColorIdentifier:
    """
    Robust color identification for robot markers using HSV space and ML
    """
    
    def __init__(self):
        # Define HSV ranges for colors (tuned for common lighting conditions)
        # Format: (lower_hsv, upper_hsv, color_name)
        self.color_ranges = [
            # Red (wraps around 0)
            (np.array([0, 70, 50]), np.array([10, 255, 255]), "red"),
            (np.array([170, 70, 50]), np.array([180, 255, 255]), "red"),
            
            # Orange (distinct from red and yellow)
            (np.array([11, 100, 100]), np.array([20, 255, 255]), "orange"),
            
            # Yellow (distinct from orange)
            (np.array([21, 100, 100]), np.array([30, 255, 255]), "yellow"),
            
            # Green
            (np.array([35, 40, 40]), np.array([85, 255, 255]), "green"),
            
            # Blue (distinct from light blue)
            (np.array([100, 100, 50]), np.array([120, 255, 255]), "blue"),
            
            # Light Blue / Cyan
            (np.array([86, 50, 50]), np.array([99, 255, 255]), "light_blue"),
            
            # Purple / Violet
            (np.array([121, 40, 40]), np.array([140, 255, 255]), "purple"),
            
            # Brown (in HSV, similar to dark orange)
            (np.array([10, 50, 20]), np.array([20, 255, 100]), "brown"),
            
            # Pink
            (np.array([140, 30, 100]), np.array([170, 100, 255]), "pink"),
            
            # White
            (np.array([0, 0, 200]), np.array([180, 30, 255]), "white"),
            
            # Black/Gray
            (np.array([0, 0, 0]), np.array([180, 30, 100]), "black"),
        ]
        
        # Color confusion matrix for post-processing
        self.confusion_pairs = {
            ("orange", "brown"): self._distinguish_orange_brown,
            ("yellow", "orange"): self._distinguish_yellow_orange,
            ("blue", "light_blue"): self._distinguish_blue_lightblue,
        }
        
    def identify_color(self, image_region):
        """
        Identify color of a robot marker region
        Returns (color_name, confidence)
        """
        if image_region.size == 0:
            return "unknown", 0.0
            
        # Convert to HSV
        hsv = cv2.cvtColor(image_region, cv2.COLOR_BGR2HSV)
        
        # Find dominant color
        color_scores = {}
        
        for lower, upper, color_name in self.color_ranges:
            # Create mask
            if color_name == "red":
                # Special handling for red (wraps around)
                mask1 = cv2.inRange(hsv, self.color_ranges[0][0], self.color_ranges[0][1])
                mask2 = cv2.inRange(hsv, self.color_ranges[1][0], self.color_ranges[1][1])
                mask = cv2.bitwise_or(mask1, mask2)
            else:
                mask = cv2.inRange(hsv, lower, upper)
            
            # Calculate percentage of pixels matching this color
            score = cv2.countNonZero(mask) / (hsv.shape[0] * hsv.shape[1])
            
            if color_name in color_scores:
                color_scores[color_name] = max(color_scores[color_name], score)
            else:
                color_scores[color_name] = score
        
        # Get top 2 colors
        sorted_colors = sorted(color_scores.items(), key=lambda x: x[1], reverse=True)
        
        if len(sorted_colors) == 0:
            return "unknown", 0.0
            
        best_color = sorted_colors[0][0]
        best_score = sorted_colors[0][1]
        
        # Check for confusion pairs
        if len(sorted_colors) > 1:
            second_color = sorted_colors[1][0]
            
            # Check if these are commonly confused colors
            pair = tuple(sorted([best_color, second_color]))
            if pair in self.confusion_pairs:
                # Use specialized distinguisher
                best_color = self.confusion_pairs[pair](image_region, best_color, second_color)
        
        # Additional features for difficult cases
        if best_score < 0.3:  # Low confidence
            best_color = self._analyze_color_features(image_region)
            
        return best_color, best_score
    
    def _distinguish_orange_brown(self, img, color1, color2):
        """Distinguish between orange and brown using brightness"""
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        
        # Orange is typically brighter than brown
        if brightness > 100:
            return "orange"
        else:
            return "brown"
    
    def _distinguish_yellow_orange(self, img, color1, color2):
        """Distinguish between yellow and orange using hue histogram"""
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hue = hsv[:, :, 0]
        
        # Yellow has higher hue values than orange
        mean_hue = np.mean(hue)
        if mean_hue > 25:
            return "yellow"
        else:
            return "orange"
    
    def _distinguish_blue_lightblue(self, img, color1, color2):
        """Distinguish between blue and light blue using saturation"""
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        saturation = hsv[:, :, 1]
        
        # Light blue typically has lower saturation
        mean_sat = np.mean(saturation)
        if mean_sat < 100:
            return "light_blue"
        else:
            return "blue"
    
    def _analyze_color_features(self, img):
        """Analyze color using multiple features for difficult cases"""
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Calculate color statistics
        mean_hue = np.mean(hsv[:, :, 0])
        mean_sat = np.mean(hsv[:, :, 1])
        mean_val = np.mean(hsv[:, :, 2])
        
        # Rule-based classification for edge cases
        if mean_sat < 30:
            if mean_val > 200:
                return "white"
            elif mean_val < 50:
                return "black"
            else:
                return "gray"
        
        # Default to hue-based classification
        if mean_hue < 15 or mean_hue > 170:
            return "red"
        elif mean_hue < 25:
            return "orange"
        elif mean_hue < 35:
            return "yellow"
        elif mean_hue < 85:
            return "green"
        elif mean_hue < 125:
            return "blue"
        else:
            return "purple"


class TemporalRobotTracker:
    """
    Main tracker combining YOLO detection with temporal smoothing and color ID
    """
    
    def __init__(self, model_path, max_age=5, min_hits=3):
        """
        Initialize tracker
        
        Args:
            model_path: Path to trained YOLOv8-pose model
            max_age: Maximum frames to keep track without detection
            min_hits: Minimum detections before track is confirmed
        """
        # Python 3.6 compatible model loading
        try:
            from ultralytics import YOLO
            self.model = YOLO(model_path)
        except ImportError:
            raise ImportError("Please install ultralytics: pip install ultralytics==8.0.0")
        
        self.max_age = max_age
        self.min_hits = min_hits
        
        # Track storage
        self.tracks = {}  # track_id -> track_info
        self.next_id = 0
        
        # Kalman trackers
        self.kalman_trackers = {}  # track_id -> KalmanTracker
        
        # Color identifier
        self.color_identifier = ColorIdentifier()
        
        # History for temporal consistency
        self.detection_history = deque(maxlen=10)
        
    def update(self, frame):
        """
        Update tracks with new frame
        
        Returns:
            List of tracks: [(track_id, bbox, orientation, color, confidence)]
        """
        # Run YOLO detection
        results = self.model(frame, conf=0.25, verbose=False)
        
        current_detections = []
        
        for r in results:
            if r.boxes is not None:
                boxes = r.boxes.xyxy.cpu().numpy()
                classes = r.boxes.cls.cpu().numpy().astype(int)
                confs = r.boxes.conf.cpu().numpy()
                
                # Get keypoints for orientation
                keypoints = None
                if hasattr(r, 'keypoints') and r.keypoints is not None:
                    keypoints = r.keypoints.xy.cpu().numpy()
                
                for i, (box, cls, conf) in enumerate(zip(boxes, classes, confs)):
                    if cls == 0:  # Robot class
                        x1, y1, x2, y2 = box
                        cx = (x1 + x2) / 2
                        cy = (y1 + y2) / 2
                        
                        # Calculate orientation from keypoint
                        orientation = 0.0
                        if keypoints is not None and i < len(keypoints):
                            kp = keypoints[i]
                            if len(kp) > 0 and kp[0][0] > 0:
                                orientation = np.arctan2(kp[0][1] - cy, kp[0][0] - cx)
                        
                        # Extract color from marker region
                        color = self._identify_robot_color(frame, box)
                        
                        current_detections.append({
                            'bbox': box,
                            'center': (cx, cy),
                            'orientation': orientation,
                            'color': color,
                            'confidence': conf
                        })
        
        # Update tracks using Hungarian algorithm
        updated_tracks = self._update_tracks(current_detections)
        
        # Add to history
        self.detection_history.append(current_detections)
        
        return updated_tracks
    
    def _identify_robot_color(self, frame, bbox):
        """Extract and identify color from robot marker region"""
        x1, y1, x2, y2 = bbox.astype(int)
        
        # Extract region above robot center (where marker is)
        height = y2 - y1
        marker_region = frame[
            max(0, y1):max(0, y1 + int(height * 0.3)),
            x1:x2
        ]
        
        if marker_region.size > 0:
            color, confidence = self.color_identifier.identify_color(marker_region)
            return color
        
        return "unknown"
    
    def _update_tracks(self, detections):
        """Update existing tracks and create new ones"""
        # Predict current positions
        for track_id, kalman in self.kalman_trackers.items():
            kalman.predict()
        
        # Associate detections to tracks
        if len(self.tracks) == 0:
            # No existing tracks, create new ones
            for det in detections:
                self._create_track(det)
        else:
            # Match detections to existing tracks
            matched, unmatched_dets, unmatched_tracks = self._associate_detections(detections)
            
            # Update matched tracks
            for det_idx, track_id in matched:
                self._update_track(track_id, detections[det_idx])
            
            # Create new tracks for unmatched detections
            for det_idx in unmatched_dets:
                self._create_track(detections[det_idx])
            
            # Mark unmatched tracks as missed
            for track_id in unmatched_tracks:
                self.tracks[track_id]['age'] += 1
                
                # Remove old tracks
                if self.tracks[track_id]['age'] > self.max_age:
                    del self.tracks[track_id]
                    del self.kalman_trackers[track_id]
        
        # Return confirmed tracks
        confirmed_tracks = []
        for track_id, track in self.tracks.items():
            if track['hits'] >= self.min_hits:
                # Get smoothed position from Kalman filter
                if track_id in self.kalman_trackers:
                    state = self.kalman_trackers[track_id].kf.statePost
                    smoothed_x = state[0]
                    smoothed_y = state[1]
                    smoothed_theta = state[4]
                    
                    # Update bbox center with smoothed position
                    bbox = track['bbox'].copy()
                    width = bbox[2] - bbox[0]
                    height = bbox[3] - bbox[1]
                    bbox[0] = smoothed_x - width/2
                    bbox[1] = smoothed_y - height/2
                    bbox[2] = smoothed_x + width/2
                    bbox[3] = smoothed_y + height/2
                    
                    confirmed_tracks.append({
                        'track_id': track_id,
                        'bbox': bbox,
                        'orientation': smoothed_theta,
                        'color': track['color'],
                        'confidence': track['confidence']
                    })
        
        return confirmed_tracks
    
    def _create_track(self, detection):
        """Create new track"""
        track_id = self.next_id
        self.next_id += 1
        
        self.tracks[track_id] = {
            'bbox': detection['bbox'],
            'center': detection['center'],
            'orientation': detection['orientation'],
            'color': detection['color'],
            'confidence': detection['confidence'],
            'hits': 1,
            'age': 0
        }
        
        # Create Kalman tracker
        kalman = KalmanTracker()
        kalman.init_state(
            detection['center'][0],
            detection['center'][1],
            detection['orientation']
        )
        self.kalman_trackers[track_id] = kalman
    
    def _update_track(self, track_id, detection):
        """Update existing track with new detection"""
        track = self.tracks[track_id]
        
        # Update track info
        track['bbox'] = detection['bbox']
        track['center'] = detection['center']
        track['orientation'] = detection['orientation']
        track['confidence'] = detection['confidence']
        track['hits'] += 1
        track['age'] = 0
        
        # Update color with temporal consistency
        if detection['color'] != "unknown":
            track['color'] = detection['color']
        
        # Update Kalman filter
        if track_id in self.kalman_trackers:
            self.kalman_trackers[track_id].update(
                detection['center'][0],
                detection['center'][1],
                detection['orientation']
            )
    
    def _associate_detections(self, detections):
        """Associate detections to tracks using IoU and color similarity"""
        if len(detections) == 0:
            return [], [], list(self.tracks.keys())
        
        if len(self.tracks) == 0:
            return [], list(range(len(detections))), []
        
        # Build cost matrix
        track_ids = list(self.tracks.keys())
        cost_matrix = np.zeros((len(detections), len(track_ids)))
        
        for d, det in enumerate(detections):
            for t, track_id in enumerate(track_ids):
                track = self.tracks[track_id]
                
                # IoU cost
                iou = self._compute_iou(det['bbox'], track['bbox'])
                
                # Color similarity
                color_sim = 1.0 if det['color'] == track['color'] else 0.5
                
                # Combined cost (lower is better)
                cost_matrix[d, t] = 1 - (iou * color_sim)
        
        # Hungarian algorithm
        from scipy.optimize import linear_sum_assignment
        det_indices, track_indices = linear_sum_assignment(cost_matrix)
        
        # Filter matches by threshold
        matched = []
        for d, t in zip(det_indices, track_indices):
            if cost_matrix[d, t] < 0.7:  # Threshold
                matched.append((d, track_ids[t]))
        
        # Find unmatched
        unmatched_dets = [d for d in range(len(detections)) 
                         if d not in [m[0] for m in matched]]
        unmatched_tracks = [track_ids[t] for t in range(len(track_ids))
                           if track_ids[t] not in [m[1] for m in matched]]
        
        return matched, unmatched_dets, unmatched_tracks
    
    def _compute_iou(self, box1, box2):
        """Compute IoU between two boxes"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0


# Python 3.6 compatible demo
def demo_tracking():
    """Demo tracking with webcam or video file"""
    
    # Initialize tracker
    tracker = TemporalRobotTracker(
        model_path="runs/pose/jetbot_orientation/weights/best.pt",
        max_age=5,
        min_hits=2
    )
    
    # Open video
    cap = cv2.VideoCapture(0)  # or video file path
    
    # Color palette for tracks
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (255, 255, 0), (255, 0, 255), (0, 255, 255),
        (128, 0, 0), (0, 128, 0), (0, 0, 128)
    ]
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Update tracker
        tracks = tracker.update(frame)
        
        # Visualize
        for track in tracks:
            track_id = track['track_id']
            bbox = track['bbox'].astype(int)
            orientation = track['orientation']
            robot_color = track['color']
            
            # Draw bbox
            color = colors[track_id % len(colors)]
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
            
            # Draw orientation
            cx = int((bbox[0] + bbox[2]) / 2)
            cy = int((bbox[1] + bbox[3]) / 2)
            ex = int(cx + 50 * np.cos(orientation))
            ey = int(cy + 50 * np.sin(orientation))
            cv2.arrowedLine(frame, (cx, cy), (ex, ey), (0, 255, 255), 3)
            
            # Draw ID and color
            label = "ID:{} {}".format(track_id, robot_color)
            cv2.putText(frame, label, (bbox[0], bbox[1]-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        cv2.imshow("Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    print("Temporal Robot Tracking System")
    print("Python 3.6 Compatible")
    print("-" * 50)
    
    # Check Python version
    import sys
    print("Python version: {}".format(sys.version))
    
    if sys.version_info[0] == 3 and sys.version_info[1] == 6:
        print("✓ Python 3.6 detected - compatible!")
    
    # Run demo
    demo_tracking()

Temporal Robot Tracking System
Python 3.6 Compatible
--------------------------------------------------
Python version: 3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


FileNotFoundError: [Errno 2] No such file or directory: 'runs\\pose\\jetbot_orientation\\weights\\best.pt'